In [38]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entire‰ly on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [39]:
# filtering out companies that don't meet basic bench marks

stock_csv = "Tickers_Example.csv"

stocks = pd.read_csv(stock_csv, header=None)

stocks.columns = ['Column1']

start_date = "2023-10-01"

end_date = "2024-09-30"

stock_list = []

count = 0
    
for tckr in stocks["Column1"]:
    stock = yf.Ticker(tckr)

    if "currency" in stock.info.keys():
        stock_currency = stock.info["currency"]
    
        stock_hist = stock.history(start=start_date, end=end_date, interval = "1mo")

        stock_volume = stock_hist.Volume.mean()

        if (stock_currency == "USD" or stock_currency == "CAD"):
            if (stock_volume.mean() > 100000):
                stock_list.append(tckr)
                #if count == 0:
                    #stock_df = pd.DataFrame({tckr : stock.history(start=start_date, end=end_date).Close})
                    #count += 1
                #else:
                    #stock_df[tckr] = stock.history(start=start_date, end=end_date).Close



stock_list

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [74]:
sector_etfs = {
    "Technology": "XLK",
    "Healthcare": "XLV",
    "Financial Services": "XLF",
    "Consumer Cyclical": "XLY",
    "Energy": "XLE",
    "Utilities": "XLU",
    "Consumer Defensive": "XLP",
    "Industrials": "XLB",
    "Real Estate": "XLRE",
    "Communication Services": "XLC"
}

# Function to get P/E ratio for sector ETFs
def get_sector_pe_data(sector_etfs):
    sector_pe_data = {}
    for sector, etf in sector_etfs.items():
        
        # Fetch ETF data
        sector_etf = yf.Ticker(etf)
        info = sector_etf.info
        
        # Get the P/E ratio of the sector ETF
        pe_ratio = info.get('trailingPE', None)
        
        if pe_ratio:
            sector_pe_data[sector] = pe_ratio
    
    return sector_pe_data

# Get sector P/E data for each ETF
sector_pe_data = get_sector_pe_data(sector_etfs)

sector_pe_data

{'Technology': 43.457508,
 'Healthcare': 24.115135,
 'Financial Services': 22.434141,
 'Consumer Cyclical': 33.346874,
 'Energy': 8.8660755,
 'Utilities': 28.26405,
 'Consumer Defensive': 27.266237,
 'Industrials': 17.707037,
 'Real Estate': 33.248898,
 'Communication Services': 36.526608}

In [73]:
def get_stock_pe(tickers):
    stock_data = {}
    for ticker in tickers:
        
        stock = yf.Ticker(ticker)
        # Get stock info
        info = stock.info
        sector = info.get('sector', 'N/A')
        pe_ratio = info.get('trailingPE', None)  # trailing P/E ratio
        pb_ratio = info.get('priceToBook', None)
        
        # Add data to dictionary
        if (sector != 'N/A'): 
            if pe_ratio:
                stock_data[ticker] = {'sector': sector, 'PE': pe_ratio}
            else:
                stock_data[ticker] = {'sector': sector, 'PE': None}
        else:
            if pe_ratio:
                stock_data[ticker] = {'sector': 'N/A', 'PE': pe_ratio}
            else:
                stock_data[ticker] = {'sector': 'N/A', 'PE': None}

    return stock_data

stock_pe = get_stock_pe(stock_list)

stock_pe

{'AAPL': {'sector': 'Technology', 'PE': 37.38962},
 'ABBV': {'sector': 'Healthcare', 'PE': 58.184673},
 'ABT': {'sector': 'Healthcare', 'PE': 35.29787},
 'ACN': {'sector': 'Technology', 'PE': 31.03937},
 'AIG': {'sector': 'Financial Services', 'PE': 14.886679},
 'AMZN': {'sector': 'Consumer Cyclical', 'PE': 42.89126},
 'AXP': {'sector': 'Financial Services', 'PE': 21.075056},
 'BA': {'sector': 'Industrials', 'PE': None},
 'BAC': {'sector': 'Financial Services', 'PE': 16.696703},
 'BB.TO': {'sector': 'Technology', 'PE': None},
 'BIIB': {'sector': 'Healthcare', 'PE': 13.925926},
 'BK': {'sector': 'Financial Services', 'PE': 17.369127},
 'BLK': {'sector': 'Financial Services', 'PE': 25.386229},
 'BMY': {'sector': 'Healthcare', 'PE': None},
 'C': {'sector': 'Financial Services', 'PE': 19.466951},
 'CAT': {'sector': 'Industrials', 'PE': 17.653275},
 'CL': {'sector': 'Consumer Defensive', 'PE': 27.091953},
 'KO': {'sector': 'Consumer Defensive', 'PE': 26.047718},
 'LLY': {'sector': 'Healthca

In [72]:
def compare_pe_to_sector(stock_data, sector_pe_data):
    # Print comparison of P/E ratios for each stock

    count_high = 0
    count_low = 0
    count_none = 0

    for tckr, data in stock_data.items():
        sector = data['sector']
        pe = data['PE']
        #pb = data['PB']
        
        if sector != 'N/A' and pe is not None:
            pe_sector_avg = sector_pe_data.get(sector, None)
            if pe_sector_avg is not None:
                if pe > pe_sector_avg:
                    stock_data[tckr]["comparisonPE"] = "higher"
                    count_high += 1
                elif pe < pe_sector_avg:
                    stock_data[tckr]["comparisonPE"] = "lower"
                    count_low += 1
                else:
                    stock_data[tckr]["comparisonPE"] = "equal"
                    count_high += 1
        else:
            stock_data[tckr]["comparisonPE"] = "None"
            count_none += 1
        
    print ("higher: " + str(count_high))
    print ("lower: " + str(count_low))
    print ("none: " + str(count_none))

compare_pe_to_sector(stock_pe, sector_pe_data)

stock_df = pd.DataFrame(stock_pe)

stock_df

higher: 12
lower: 22
none: 3


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
sector,Technology,Healthcare,Healthcare,Technology,Financial Services,Consumer Cyclical,Financial Services,Industrials,Financial Services,Technology,...,Technology,Financial Services,Technology,Communication Services,Financial Services,Technology,Healthcare,Industrials,Industrials,Financial Services
PE,37.372322,58.161327,35.28441,31.027945,14.882703,42.867805,21.077263,None,16.692028,None,...,17.23602,15.052212,96.079994,34.174606,18.030092,36.721706,38.594414,21.361574,19.834986,15.580677
comparisonPE,lower,higher,higher,lower,lower,higher,lower,None,lower,None,...,lower,lower,higher,lower,lower,lower,higher,higher,higher,lower


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.